# FS-3693 Address Lookup Spike
Spike to check we can connect to the post code lookup on OS GB places API, and then also if we can reconcile that with the local links manager local authority data.

## Postcode search tech docs
https://osdatahub.os.uk/docs/places/technicalSpecification

## Local links manager
https://github.com/alphagov/local-links-manager/blob/main/data/local-authorities.csv

## Setup
1. Create a virtual env `python3 -m venv .venv`
1. Activate the venv `source .venv/bin/activate`
1. Install requirements using `pip install -r requirements.txt`

In [1]:
# Uses the dotenv extension to load vars from file .env
%load_ext dotenv
%dotenv

# Retrieve data from postcode lookup
Uses the api key (in bitwarden - add it to environment) to retrieve all addresses within the specified postcode, using the supplied language

In [2]:
import requests, json, os
api_key = os.getenv("PLACES_API_KEY")
postcode = "RG29UD"
lang = "en"
url = f"https://api.os.uk/search/places/v1/postcode?key={api_key}&postcode={postcode}&lr={lang}"
results = requests.get(url=url)
if results.status_code == 200:
    result_json = results.json()
elif results.status_code == 401:
    print("Unauthorised")
else:
    error_message = json.loads(results.text)["error"]["message"]
    print(f"Error retrieving data: {error_message}")


# Print search results

## Print full addresses

In [3]:
results_count = result_json["header"]['totalresults']
if results_count == 0:
    print("No results found")
else:
    print(f"There are {results_count} results")
    for r in result_json["results"]:
        print(r["DPA"]["ADDRESS"])

There are 6 results
FLAT 1, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD
FLAT 2, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD
FLAT 3, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD
FLAT 4, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD
FLAT 5, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD
FLAT 6, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD


## Print a single record

In [4]:
record = result_json["results"][0]
print(record)

{'DPA': {'UPRN': '10024050924', 'UDPRN': '54779187', 'ADDRESS': 'FLAT 1, HIRST HOUSE, 15, QUILLER AVENUE, ARBORFIELD GREEN, READING, RG2 9UD', 'SUB_BUILDING_NAME': 'FLAT 1', 'BUILDING_NAME': 'HIRST HOUSE', 'BUILDING_NUMBER': '15', 'THOROUGHFARE_NAME': 'QUILLER AVENUE', 'DEPENDENT_LOCALITY': 'ARBORFIELD GREEN', 'POST_TOWN': 'READING', 'POSTCODE': 'RG2 9UD', 'RPC': '1', 'X_COORDINATE': 476351.23, 'Y_COORDINATE': 165022.95, 'STATUS': 'APPROVED', 'LOGICAL_STATUS_CODE': '1', 'CLASSIFICATION_CODE': 'RD06', 'CLASSIFICATION_CODE_DESCRIPTION': 'Self Contained Flat (Includes Maisonette / Apartment)', 'LOCAL_CUSTODIAN_CODE': 360, 'LOCAL_CUSTODIAN_CODE_DESCRIPTION': 'WOKINGHAM', 'COUNTRY_CODE': 'E', 'COUNTRY_CODE_DESCRIPTION': 'This record is within England', 'POSTAL_ADDRESS_CODE': 'D', 'POSTAL_ADDRESS_CODE_DESCRIPTION': 'A record which is linked to PAF', 'BLPU_STATE_CODE': '2', 'BLPU_STATE_CODE_DESCRIPTION': 'In use', 'TOPOGRAPHY_LAYER_TOID': 'osgb5000005254619208', 'PARENT_UPRN': '10024050914', 

## Display derived information
Displays the devolved nation name from a simple lookup, and the local authority using the local links manager local authority data and local custodian code.

In [5]:
# Load lookups
countries = {
    "W": "Wales",
    "N": "Northern Ireland",
    "E": "England",
    "S": "Scotland"
}


In [6]:
local_custodian_code = str(record['DPA']['LOCAL_CUSTODIAN_CODE'])
country_code = record['DPA']['COUNTRY_CODE']

print(f"Devolved nation: {countries[country_code]}")

# Search CSV file for local custodian code and find local authority name
import csv
with open("local-authorities.csv", mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row["local_custodian_code"] == local_custodian_code:
            print(f"Local authority: {row['name']}")
            break

Devolved nation: England
Local authority: Wokingham Borough Council


# Notes for implementation
- Ask user to enter postcode (anything from the first 3 characters works with this API, eg. RG2)
- Validate it's part of a postcode before search
- Display results and let them choose.
- Country code is in results and lets us know their devolved nation.
- To find local authority name, use local custodian code and search local links data for that.
- If they edit or enter address manually:
    - Display country (E/NI/S/W) as drop down so they don't enter UK
    - Display local authority as drop down